# Tarefa 2 — Random Forest (RF)

## Passo a passo do método

- <b>Passo 1</b> — Bootstrap (amostras com reposição)
A partir do conjunto de treino, são geradas várias amostras bootstrap (com reposição). Isso cria vários subconjuntos ligeiramente diferentes dos dados.

- <b>Passo 2</b> — Feature selection (seleção aleatória de variáveis)
Para introduzir diversidade entre as árvores, o Random Forest seleciona um subconjunto aleatório de features para considerar nas decisões. Na ideia clássica do RF, essa seleção acontece em cada divisão (split) da árvore.

- <b>Passo 3</b> — Modelagem com Decision Trees
Para cada amostra bootstrap, é treinada uma árvore de decisão. No final, forma-se uma “floresta” com várias árvores.

- <b>Passo 4</b> — Agregação (combinar previsões)
As previsões das árvores são combinadas em uma única saída:

Classificação: votação (classe mais frequente entre as árvores).

Regressão: média das previsões numéricas.

## O que é Random Forest?

Random Forest é um método de ensemble que cria várias árvores de decisão e combina as previsões delas para obter um resultado final mais estável. Ele se parece com Bagging porque também usa bootstrap e agregação, mas adiciona um ponto essencial: além de variar os dados (via bootstrap), ele também varia as features consideradas, o que reduz ainda mais a chance de várias árvores ficarem “parecidas” e melhora a robustez do modelo.

## Diferença entre Bagging e Random Forest

Bagging: treina vários modelos (geralmente árvores) usando bootstrap e combina as previsões. A diversidade vem principalmente da variação nos dados.

Random Forest: faz tudo que o Bagging faz e adiciona seleção aleatória de features durante a construção das árvores. Isso aumenta a diversidade entre elas e costuma melhorar o desempenho/estabilidade.

# Exemplo do modelo — Random Forest em Python

Abaixo vai um exemplo manual didático (mostra claramente: Bootstrap → Feature selection → Árvores → Agregação) e depois a versão oficial com sklearn.

In [8]:
import pandas as pd
import numpy as np
import pandas as pd


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer

## 1) Carregar uma base pronta (breast cancer)

In [6]:
data = load_breast_cancer(as_frame=True)
df = data.frame.copy()

print("Shape do df:", df.shape)
df.head()


Shape do df: (569, 31)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


## 2) Exempolo Random Forest 

Observação: aqui a feature selection é feita por árvore (subconjunto de colunas por árvore) para ficar simples e bem visível. No RF “completo”, o sorteio de features ocorre a cada split.

In [12]:
# Definindo X e y
target = "target"
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# Random Forest fiel (bootstrap + max_features por split + votação)
rf = RandomForestClassifier(
    n_estimators=200,
    bootstrap=True,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

acc = accuracy_score(y_test, y_pred)

res = pd.DataFrame({
    "y_real": y_test.reset_index(drop=True),
    "y_pred": pd.Series(y_pred)
})

print(f"Acurácia (Random Forest): {acc:.4f}")
print("Linhas no resumo:", len(res))
res.head()

Acurácia (Random Forest): 0.9415
Linhas no resumo: 171


,y_real,y_pred
0,0,0
1,1,1
2,1,1
3,0,0
4,0,1
